In [2]:
import pydicom
import torch
import os
import sys
import pandas as pd
import numpy as np
import pydicom as DCM
import png
import matplotlib.pyplot as plt
from __future__ import print_function, division
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image, ImageOps, ImageEnhance
from sklearn.preprocessing import LabelEncoder


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from skimage import img_as_float

In [4]:
import warnings
warnings.filterwarnings("ignore")
plt.ion()

In [ ]:
# cwd = os.getcwd()
# cwd

In [ ]:
# landmarks_frame = pd.read_csv('faces/face_landmarks.csv')
# n = 65
# # accessing nth row 
# img_name = landmarks_frame.iloc[n, 0]
# landmarks = landmarks_frame.iloc[n, 1:].as_matrix()
# landmarks = landmarks.astype('float').reshape(-1, 2)
# # unknown rows but only two columns x and y

# print('Image name: {}'.format(img_name))
# print('Landmarks shape: {}'.format(landmarks.shape))
# print('First 4 Landmarks: {}'.format(landmarks[:4]))
# plt.imshow(io.imread(os.path.join('faces/',img_name)))

In [8]:
def createTrainFrame():
    
    train_mass_csv = pd.read_csv("../../CuratedDDSM/Train/mass_case_description_train_set.csv")
    train_calc_csv = pd.read_csv("../../CuratedDDSM/Train/calc_case_description_train_set.csv")
    train_calc_csv = train_calc_csv.rename(columns={'breast density': 'breast_density'})
    train_calc_csv['image file path'] = 'Calc/CBIS-DDSM/'+ train_calc_csv['image file path']
    train_calc_csv['ROI mask file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['ROI mask file path']
    train_calc_csv['cropped image file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['cropped image file path']
    train_mass_csv['image file path'] = 'Mass/CBIS-DDSM/' + train_mass_csv['image file path']
    train_mass_csv['ROI mask file path'] ='MassROI/CBIS-DDSM/' + train_mass_csv['ROI mask file path']
    train_mass_csv['cropped image file path'] ='MassROI/CBIS-DDSM/'+ train_mass_csv['cropped image file path']
    common_col = list(set(train_calc_csv.columns) & set(train_mass_csv.columns))
    train = pd.concat([train_mass_csv[common_col], train_calc_csv[common_col]], ignore_index=True,sort='False')
    train['image file path'] = 'CuratedDDSM/Train/'+train['image file path'] 
    train['ROI mask file path'] = 'CuratedDDSM/Train/'+train['ROI mask file path']
    train['cropped image file path'] = 'CuratedDDSM/Train/'+train['cropped image file path']
    train['pathology_class'] = LabelEncoder().fit_transform(train_df['pathology'])
    
    return train

def createTestFrame():
    test_mass_csv = pd.read_csv("../CuratedDDSM/Test/mass_case_description_test_set.csv")
    test_calc_csv = pd.read_csv("../CuratedDDSM/Test/calc_case_description_test_set.csv")
    test_calc_csv = test_calc_csv.rename(columns={'breast density': 'breast_density'})
    test_calc_csv['image file path'] = 'Calc/CBIS-DDSM/'+ test_calc_csv['image file path']
    test_calc_csv['ROI mask file path'] ='CalcROI/CBIS-DDSM/' + test_calc_csv['ROI mask file path']
    test_calc_csv['cropped image file path'] ='CalcROI/CBIS-DDSM/' + test_calc_csv['cropped image file path']
    test_mass_csv['image file path'] = 'Mass/CBIS-DDSM/' + test_mass_csv['image file path']
    test_mass_csv['ROI mask file path'] ='MassROI/CBIS-DDSM/' + test_mass_csv['ROI mask file path']
    test_mass_csv['cropped image file path'] ='MassROI/CBIS-DDSM/'+ test_mass_csv['cropped image file path']
    common_col = list(set(test_calc_csv.columns) & set(test_mass_csv.columns))
    test = pd.concat([test_mass_csv[common_col], test_calc_csv[common_col]], ignore_index=True,sort='False')
    test['image file path'] = 'CuratedDDSM/Test/'+test['image file path'] 
    test['ROI mask file path'] = 'CuratedDDSM/Test/'+test['ROI mask file path']
    test['cropped image file path'] = 'CuratedDDSM/Test/'+test['cropped image file path']
    train['pathology_class'] = LabelEncoder().fit_transform(train_df['pathology'])
    
    return test

In [9]:
train_df['class'] = LabelEncoder().fit_transform(train_df['pathology']) 
train_df[['pathology', 'class']]
    

NameError: name 'train_df' is not defined

In [11]:
train_df = createTrainFrame()
test_df = createTestFrame()
mammogram_dir = '/home/himanshu/CuratedDDSM/'
train_df.to_csv('train.csv')
test_df.to_csv('test.csv')
pd.get_dummies(train_df, columns=['pathology'])
# pytorch handles numeric labels for inbuilt loss functions
train_df

NameError: name 'train_df' is not defined

In [4]:
train_df = createTrainFrame()

FileNotFoundError: File b'../CuratedDDSM/Train/mass_case_description_train_set.csv' does not exist

In [ ]:

# image_path = os.path.join(homedir,train['image file path'][0])
# image_path

In [ ]:
# train['image file path'][0]

In [ ]:
# train = pd.concat([train_calc_csv[common_col], train_mass_csv[common_col]], ignore_index=True,sort='False')
# train['image file path'] = 'CuratedDDSM/Train/'+train['image file path'] 


In [ ]:
labels = set(train_df['pathology'])
labels

In [ ]:
# a =  DCM.read_file('../CuratedDDSM/Train/Calc/CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC/08-07-2016-DDSM-23157/1-full mammogram images-38548/000000.dcm')

In [ ]:
# index=0
# image = DCM.read_file('../CuratedDDSM/Train/'+train_mass_csv['image file path'][index])
# label = train_mass_csv['pathology'][index]
# print(image.pixel_array.shape)
# image_2d = image.pixel_array.astype(float)
# # plt.imshow(image_2d,cmap='gray')
# image_2d_scaled = (np.maximum(image_2d,0)/ image_2d.max()) * 255.0
# image_2d_scaled = np.uint8(image_2d_scaled)
# # plt.imshow(image_2d_scaled,cmap='gray')
# transformations = \
#             transforms.Compose([transforms.Grayscale(3),
#                                transforms.Resize(512,interpolation=Image.LANCZOS),
#                                transforms.CenterCrop(512)])
# img = Image.fromarray(image_2d_scaled)
# img = transformations(img)
# print(img.getpixel)
# img_np = np.asarray(img)
# plt.imshow(img_np)
# img_np.shape

In [ ]:
class MammographyDataset(Dataset):
    """Creating CBIS-DDSM pytorch dataset."""

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file containing labels.
            root_dir (string): path to CuratedDDSM directory.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # Transforms
        self.to_tensor = transforms.ToTensor()
        # Read the csv file
        self.frame = pd.read_csv(csv_file)
        # Image Columns
        self.image_arr = np.asarray(self.frame['image file path'])
        # Labels
        self.label_arr = np.asarray(self.frame['pathology'])
        # Calculate Len
        self.data_len = len(self.frame.index)
        # Location of Curated DDSM
        self.root_dir = root_dir
        # Transformations to convert image into 512*512
        self.transformations = \
            transforms.Compose([transforms.Grayscale(1),
                               transforms.Resize(512,interpolation=Image.LANCZOS),
                               transforms.CenterCrop(512)])

    def __len__(self):
        return self.data_len

    def __getitem__(self, index):
        # Open image
        #img_as_img = Image.open(single_image_name)
        image_path = os.path.join(self.root_dir,self.image_arr[index])
        image_dcm= DCM.read_file(image_path)
        
        image_2d = image_dcm.pixel_array.astype(float)
        image_2d_scaled = (np.maximum(image_2d,0)/ image_2d.max()) * 255.0
        image_2d_scaled = np.uint8(image_2d_scaled)
        #Transform image to 512 * 512 
        img = Image.fromarray(image_2d_scaled)
        img_as_img = self.transformations(img)
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        image_label = self.label_arr[index]

        return (img_as_tensor, image_label)

In [ ]:
# import numpy as np
# import png
# import pydicom

# ds = pydicom.dcmread(path)

# shape = ds.pixel_array.shape

# # Convert to float to avoid overflow or underflow losses.
# image_2d = ds.pixel_array.astype(float)

# # Rescaling grey scale between 0-255
# image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

# # Convert to uint
# image_2d_scaled = np.uint8(image_2d_scaled)

# # Write the PNG file
# with open(destination, 'wb') as png_file:
#     w = png.Writer(shape[1], shape[0], greyscale=True)
#     w.write(png_file, image_2d_scaled)